In [1]:
import requests
import hashlib
from datetime import datetime
import time
import random
import os
import csv
from bs4 import BeautifulSoup
import re 
import pandas as pd

# Parsing functions

In [2]:
def parse_listing_html(text,file_name):
    #define the html text
    page_html = BeautifulSoup(text, 'html.parser')
    
    if deleted_listing(page_html):
        print('listing {} has been deleted'.format(file_name))
        return
    
    if page_html.find('meta',attrs={'property':'og:url'}) is None: 
        return
    else:
        listing_url = page_html.find('meta',attrs={'property':'og:url'}).attrs['content']
    try:
        listing_id = page_html.findAll('p', class_='postinginfo')[1].text[9:]

        listing_title = page_html.find('span',attrs={'id':'titletextonly'}).text
        
        listing_nh = page_html.find('small').text[2:-1]
        
        listing_city = page_html.find('meta',attrs={'name':'geo.placename'}).attrs['content']
        
        listing_date = page_html.find('time', class_='date timeago')['datetime']
        
        if page_html.find('span', class_='price') is None: 
            return
        else:
            listing_price = int(page_html.find('span', class_='price').text.strip('$,').replace(",",""))

        
        listing_bedrooms = page_html.find('span', class_='shared-line-bubble').text.split('BR')[0]
        
        listing_bathrooms = page_html.find('span', class_='shared-line-bubble').text.split('/')[1][:-2]
        
        # Extract sqft info from bubbles class and use regex to find the digits ending in ft2
        bubbles_sqft = page_html.findAll('span', class_='shared-line-bubble')
        listing_sqft = None
        if len(bubbles_sqft) >= 2:
            x = re.search("[0-9]+ft2$", bubbles_sqft[1].text)
            if x:
                 listing_sqft = x.string[:-3]

        listing_address_element = page_html.find('div', class_='mapaddress')
        listing_address = listing_address_element and listing_address_element.text

        listing_info = page_html.findAll('p', class_='attrgroup')[1].text

        listing_body = page_html.find('section',attrs={'id':'postingbody'}).text[30:]

        listing_first_image = page_html.find('div',class_='slide first visible').img['src']
    except Exception as e:
        print('Error parsing url: {} file: {}'.format(listing_url, file_name))
        raise e

    return {
        'listing_title': listing_title,
        'listing_nh': listing_nh,
        'listing_city':listing_city,
        'listing_date':listing_date,
        'listing_price': listing_price,
        'listing_bedrooms':listing_bedrooms,
        'listing_bathrooms':listing_bathrooms,
        'listing_sqft':listing_sqft,
        'listing_address':listing_address,
        'listing_info':listing_info,
        'listing_body':listing_body,
        'listing_id':listing_id,
        'listing_url':listing_url,
        'listing_first_image':listing_first_image
       }


def deleted_listing(page_html):
    h2 = page_html.find('h2')
    return h2 and h2.text == ' This posting has been deleted by its author. '

# Create df from dictionary

In [3]:
list_of_dicts = []
path = "/Users/pandabear/springboard/CapstoneTwoProject/data/raw"
dir_list = os.listdir(path)


### Testing parsing for one folder scraped by playwright
# path_testing = '/Users/pandabear/springboard/CapstoneTwoProject/data/raw/2022-08-17T22:01:42'
# file_list = os.listdir(path_testing)
# for file in file_list:
#     if len(file) > 15:
#         with open(path_testing + '/' + file, 'r') as f:
#             text = f.read()
#             parsed_data = parse_listing_html(text, file)
#             if parsed_data:
#                 list_of_dicts.append(parsed_data)

# listing_df = pd.DataFrame(list_of_dicts)
### Testing code


for folder in dir_list:
    if not folder.startswith('.'):
        path_html = path + '/' + folder
        file_list = os.listdir(path_html)
#         print(file_list)
        
        for file in file_list:
            if len(file) > 15:
                with open(path_html + '/' + file, 'r') as f:
                    text = f.read()
                    parsed_data = parse_listing_html(text, file)
                    if parsed_data:
                        list_of_dicts.append(parsed_data)
#                 print('Number of parsing errors: {}'.format(count_parsing_error))
        new_listing_df = pd.DataFrame(list_of_dicts)


listing a660d34c3d055751c7c75cf53c5ec04cb7ec75343f06972eebdef11309cf1117 has been deleted
listing 48fc42305b6e0e673ade36447492f663be18398e23158fb50b3e94ea2427091c has been deleted
listing 18198c7320b10e1720c5a2812c2e15849aad58ad73e5ede4e358ce54e11d018b has been deleted
listing b75bf9e6ee658dfd32c6ece25e269bf2daa73584b02bdfa911dd5ae78cb5df10 has been deleted
listing 6f88f71f66764367d67d00a72bd77f3b978628add5a39b219dbb616e19b2ef99 has been deleted
listing 930890b337519252e56ce2b72cf32c17eefa1f57d39866af018ae34da5764511 has been deleted
listing f4e41927ac05269751859baab5ec607f80ffd2618032718a3cef1486dab5a449 has been deleted
listing 6110badfd66e672449330d3dfa1128b886dc4da6316578328ee482acc91bcba5 has been deleted
listing 7125827eff6437b0b6094843c8d806df2b3855faa7590d477ba970cf81c78e4f has been deleted
listing b1b4dc7aca16e69e2af587206e8ab68bcf012f3443ee2b702df6da8374c3a88c has been deleted
listing 8a1aadc1e5964bc98d50ed18ae30dc1dc10b68bb545ec4749d4ab2fd76f36650 has been deleted
listing b8

listing 1e3d5a1eddfaeb8f5cf7d8e1509361d07ef0132c489b5eff9fa49828ca41e9e8 has been deleted
listing 791deba6efcb4b425719574cbc7b3e132f7cf5e53f04d687d0ffb525df89816f has been deleted
listing fd01ed5872de4344a97b5c516865cdff1c3c58097d37cce7ed2a5474acaad309 has been deleted
listing 86129e517a2613b0dc196249c49c2ba5673607b97fd46f46b9f4f42adb5809a3 has been deleted
listing 103c3cca54753dfc5f9696f35ead6a94dfe71c1b32315ef95448592d81e735ea has been deleted
listing c2b2f910b98fc0ef109fa86c1a9d43b11c38dcd9a5408f362281ea73a8a0e5d2 has been deleted
listing 7281ca4355fe5d4b5015617cff14efaffcec8731f4a6dea426cccd92b6263eec has been deleted
listing d24d019b754ed31ecfa1eadbd0a3251f148439371a3c556273ae487a46f33dcd has been deleted
listing 78c8d6d5d03a33111ab52cc497873a0daefcddb93524f07ca0ef56ae9392f780 has been deleted
listing b951e656cd35634c5c7358a5ae454830fe09941f4538628f250c46a69ec70e3e has been deleted
listing dd1b32b71c6b62c6c64e4965c4cf954cb8545d1511229c9db0418615929b513d has been deleted
listing c7

listing c3ebbc8374a4a0367c33930b9febd2912bcc50ccfa81c45072ae53949ac2ad43 has been deleted
listing 95f54c7e80ec2ae6f2bdf7371b3b2dea03330e002cf921a8d19a1fdc820a4a79 has been deleted
listing 5f5d6aa9c2a056b916dfcbb535378b33b576991f97f284c3cb406700ce8e61b0 has been deleted
listing 24a300fb37daf5d4d8cb67a405c34b6400725e5155947d91802c93d6f931c226 has been deleted
listing 0da55b4c7bc2af1f9c0497dcfbfbefaf70be6823798f6b1bb7559f1119cf5bff has been deleted
listing 1c7d466e26470911970082b8ff98f8d56c73fb1dcc12b4832b09cf65c6558a6c has been deleted
listing a44960338939555bdda94f5f36b638db95f40591d5d404835184b779b148c27c has been deleted
listing c66c6f3463892225ded092809ce2eaa19f6492788739e6545849dc7175baceea has been deleted
listing cf65b41e50cf39c4968b016513bb6918bcbbe86e08a8785e018ed9642d01902f has been deleted
listing 936d12ecf2c09bd11ee243acd3cbba689506512ad1bfde5b184bf87c4a03565d has been deleted
listing bf123a3c60778b5f169520a2e98678408e2b6be892ed09928f3fab8c7a3563b7 has been deleted
listing 8a

listing a3bf59fe21c359ec092da5beaee88469f7fe834ffd81a0e2bb7fc136bbca7a12 has been deleted
listing 760d51fb3efe5fdeeb4f539e89fd7cea58ebcdc6a4bb05f90d56f24c4df49189 has been deleted
listing f3adba5ea0207671bc1c0a9cdbf3a8210fd65b5ed76455440d11fcbcdcd1a8d4 has been deleted
listing 9b64b0dfe9e24db673ca6e0dddbec55adc4dfd89c9c30294e372802bad726086 has been deleted
listing af67244eba92f2e371c8d92d9cefd561be000d68412366ca4577d29cbead3cc1 has been deleted
listing 2b32da9603eafb73243ad858b25c3b85bb73dfd831b2ac649fad54656ac9914c has been deleted
listing f12f247b75edcdca1959016ce2d2c27e2eee956098618a48f01d4a483636f7de has been deleted
listing c176912e21395ff7a12977a44d32d9f25045f922e430009a5dedbfff21196e78 has been deleted
listing 5e30c2841520e77a561b8d3a2d24189eac2d6cf7266843023eabf8f8cf0f1940 has been deleted
listing 1097e2c9a1a4a4c92bcf208477d9c45e52e624c4048dac7a150c59a593314a9e has been deleted
listing 2d592df408593b8a53830e04d4bcbe515dc85981b870f899aeaf74bd7844790b has been deleted
listing c2

listing b58bc73e5d5ee5137958a45453b2a54fd86aba0825945e467c0651ffcde6bf6a has been deleted
listing 20685a7e6b65f30d926bc83fdadf0b891f30831eefcec05845fca8b00e53ae7c has been deleted
listing a5f95b59ce094672e40caf1c851f5bb3c3604f347ff68baf5566b7687d0cd945 has been deleted
listing c79209589ef3fb8cd0e799318638ad0ea77427e15f5dc70d53b152233bf778a3 has been deleted
listing b676531875e78aee7a3539ff0e61c96f5279b1e328a39fa7394f7396fa5bc8e8 has been deleted
listing f049255a67b7a3747ed54d5603a0f7e99b8e3fae40c81235842f4d4cf2e72088 has been deleted
listing 47e25ded0f831643c898250cba4431cf5612b88418caa032a666950421011738 has been deleted
listing 43b33b6f2d6021aace479120beb137ce0b7479787125d3d19541ab21db5d1a03 has been deleted
listing 95fc7c00397e9ac7432b283eb98d8beb6bf801c9b070c2b26a171dd4bcd6e6d5 has been deleted
listing 04fe86ece7356d31709b37d485064abcc4199fde867f097eae1b5f10001a3c52 has been deleted
listing e4a9ebe1c24cafc67953791a1590c808390333941508e2199e3e7b5667719503 has been deleted
listing 2b

...

listing e747c6ea446599b9db184002d47239c5c8d3e79035bbe6652362962e26ae7295 has been deleted
listing 0b60f292d1872c16ef93d612c46a80af72989ed3ec1f113b3d67df3df213c4ef has been deleted
listing 5843eef3aecda27fa45aa695c96cf057d295f429ba7fbfce5ea4b63641851db7 has been deleted
listing c39de4abc135786e983ab7598584925497ac4dc0bc644981c911fe8d99f1d630 has been deleted
listing e1ab40ed76458018182c72fc79c16df91ef2ce79d63321a7dff71147e4fcb291 has been deleted
listing ad066a51869537e3ad82b57f4101e16a7bf8190433dcd6946bd30c055f57ec22 has been deleted
listing 5492b046c6937fc437031916488a6614748a1be88cd2c58e9a85d5a4f973e8f9 has been deleted
listing 40a29d0d0bddd3d997c4c493cbd3295024ef62c8c4bb3f077069582517367984 has been deleted
listing 80a83d5789b39dbef539923977a6fd2b9f02a53ad2e2723a04a2c44184920503 has been deleted
listing ad246bf1e93e6f8dd688bf7d5709cd1fa3cffdc276e7f0dd2ac514d450aa9bf4 has been deleted
listing 26e114a095086ade001ed2372af055b1c4bcb73009caa3b16ce0ec73e1264b8f has been deleted
listing e6

listing 512122fcfc0b204399d5c3316d3167d47ce443d8113da51ea922ea10aaff6706 has been deleted
listing 025418cd74dac66a9f77dceb9e7bcc56af6fcb1a55b591178704c1f34c555763 has been deleted
listing 8aca38b2539fddbc5099a01211646323ab5e458174e07be206ed6aa807682db8 has been deleted
listing 1214343c4b72ad25dc103858080bd2e09ed33db9ad8c6f9bf0b421e8a4422329 has been deleted
listing b5c6bdc00c0a4680743b177be29ae1896507c4e08d2335855f4adb9be3cdbe47 has been deleted
listing e1ca4f9272d6e72430ac6d107de5c34149648a8c4b821db497a4e6c739e259cd has been deleted
listing 842f1ab4ea62e02b8b12634b22b6c8ac8699e294b0d0a2d3304eaeb5d741b1a6 has been deleted
listing 5a954cd2154896713d33cb9d8f357ac62aee9607d0c21d6044ac87959014f1e2 has been deleted
listing 0f2cc33db0dcd64deae2ae4f63ef0fca060e0d84051f900489129612dcd84724 has been deleted
listing 5c5a8a01fb82d38b538851724de172ceb61b121c7922dba752a329b9756bd594 has been deleted
listing e8e2788092ab81fe0b791f6cbc58567eb1afaa5be032475d96b5b17d513d87a6 has been deleted
listing 83

listing aba831319080c1d3824706c36587a075f989b296a8855be9f6a87d0c0e138515 has been deleted
listing 670a5bcb510cc708ad4557ab221b67a53c0ca6b4a73a16aa3666509e07abeaa0 has been deleted
listing d389328179bd3c5b7cd6a4d52dcf3f0b244343fdc7a7c0b742697c9b16043d63 has been deleted
listing bea60d38e74cc096f53b28a2d3a62e00bff00c438d205057c41a3f7ddf5b34b4 has been deleted
listing 7d845afb0f593daeeac2630009bc937f8c1af218a57f38a22397a8f8df04a67b has been deleted
listing 0e92dd5a91edbc415066d7b648b575913936d2eae9ecc3cda2645653e33c5dab has been deleted
listing 37ccae84638d48b016b02bd3c6ceb217cf3887f6f6cdfb3db95d0d17aa42ed55 has been deleted
listing 3da3969bc85a8acce5ffa591d06a28c031e55fedb741f28f5dc05cadca75f55e has been deleted
listing 2ebde0cce7db70a90e6978da94ae4054503da8aa2804fbfcfc3927c5ea89312d has been deleted
listing 76691677773a0db882b80ca8dd879e7aae26637892e7666bafd95427f9517447 has been deleted
listing 410ef240a74c0fde3095b765043939819041e1a2a8b71fefbe0ef43216783328 has been deleted
listing 95

listing 3a02945ce86bd2f0f38583aaf742fd82fa309587a2d05ec10ce22971c392484d has been deleted
listing 9dda20949d6e4f89d113bb03c26da05b16ee99f54b8772efde53eb69fb22e9e6 has been deleted
listing 8764271559c05b76b1eac4ebc216b7855392331142746df0b1164c7ea3d92ebc has been deleted
listing 729ed0928476f0d2d6e9a7856b6a996e7a19c91365a4cd496f38cf401db9c2e5 has been deleted
listing de11741065ddedec0cce4d217913af49880a4391f4dc4e13fb210f3bb8b52318 has been deleted
listing 04ea2f3ebb268e5f094b07bb637fa67d7c076107ccae9a1e214367c8e3eb65e0 has been deleted
listing ce0bc8ac33b0c78b07482ab63ee61bc1d91eb051708c127d6f22cea5c37a9511 has been deleted
listing c04462c452fabd07f4da105209a26876367675ccd68e5e51804f511745990921 has been deleted
listing 7607fc5752c8efa2dfd8c9cf60e1b00b6bda54ab22f8cfb69b93ed587035cec7 has been deleted
listing bbf1106993f3edd8a50dd85cfd4869c8440f693d4e02d7eff879d44960961d83 has been deleted
listing 783982260425b63429c6840b6aa1b9e1b3cf382428df42ed2363a388d7fda926 has been deleted
listing 5e

listing 90752ae083c31e4bf1a1a5ee47ee9b73c817c7bad8321e2e32952807fb762488 has been deleted
listing 7f5860fe676cd1d9603cb4efa3f7f2527e9d79912f11f3e2db2214618fed5178 has been deleted
listing 3a0f2646afee93003c24da7479bf46616ccfe844a3477b7f4fefc5b175b5d5ed has been deleted
listing d87e792ba5b702e53501fc1ebc90c9c837434d22997432a5a7cc9c178aa0769a has been deleted
listing c99dbb19686ccd9927b60c2e553d1c463b88ca29984dab49847848e50865eebf has been deleted
listing a643a679b24ed8b3c380d362617326ca568cd424da5d8baac0867136f179cf7f has been deleted
listing cdb90b387f978ce64346bfe0776cb46722d7af616def9bddb596e4d553148772 has been deleted
listing 3478081343f8ceabd20acd14c84be74cf5d219a8a158c314b221fdaf01bc8709 has been deleted
listing d03f20118224630f5915e73a4e14515adf31f4a6e50334aa64f1db66896c1d6f has been deleted
listing 8c97d99cd8f228662aec39c9d128c9ae81527873e6e0281073137e5fc3001dbb has been deleted
listing e63d4351ead7b48167a45c416797af039baac03df2df09f0b9bad10b1c936ced has been deleted
listing a7

In [4]:
new_listing_df.set_index('listing_id', inplace = True)
new_listing_df.head()

,listing_title,listing_nh,listing_city,listing_date,listing_price,listing_bedrooms,listing_bathrooms,listing_sqft,listing_address,listing_info,listing_body,listing_url,listing_first_image
listing_id,,,,,,,,,,,,,
7544341056,Avail of the limited offer - 2 MONTHS FREE!,south san francisco,South San Francisco,2022-10-11T09:22:06-0700,3870,1,1,692,988 El Camino Real,EV charging air conditioning cats are OK - ...,Call Now - show contact info x 116OR Text 116 ...,https://sfbay.craigslist.org/pen/apa/d/south-s...,https://images.craigslist.org/00505_klHelxMNBn...
7533119488,Welcome to the Central Park neighborhood of Sa...,san mateo,San Mateo,2022-09-12T15:52:15-0700,2900,2,1,750,612 s el camino real,application fee details: $41.99 application f...,Welcome to the Central Park neighborhood of Sa...,https://sfbay.craigslist.org/pen/apa/d/san-mat...,https://images.craigslist.org/00707_jLLpHEtFm9...
7546896831,💡 Beautiful Location! 2BED/BATH available NOW!,palo alto,Palo Alto,2022-10-17T18:01:59-0700,3450,2,2,1000,3375 Alma Street,open house dates tuesday 2022-10-18 thur...,To schedule a tour We now book our tour appoin...,https://sfbay.craigslist.org/pen/apa/d/palo-al...,https://images.craigslist.org/00h0h_wi7SjXA3VO...
7542991210,CALL NOW 1 MONTH FREE - 2 BEDROOM APARTMENT,redwood city,Redwood City,2022-10-07T15:23:41-0700,2995,2,2,900,1887 woodside rd,application fee details: $35 application fee ...,FEATURES Bedrooms: 2 Bathrooms: 2 Located on F...,https://sfbay.craigslist.org/pen/apa/d/redwood...,https://images.craigslist.org/00s0s_6YYC7esZDK...
7548286030,"Great location! Minutes to shopping, Bart, Cal...",burlingame,Burlingame,2022-10-21T09:12:32-0700,2495,1,1,None,1830 Sequoia Ave near Murchinson Drive,cats are OK - purrr dogs are OK - wooof apa...,Welcome Home to Classic Peninsula Living at Bu...,https://sfbay.craigslist.org/pen/apa/d/millbra...,https://images.craigslist.org/00202_6KfgOHGKET...


In [5]:
new_listing_df.shape

(299576, 13)

In [6]:
# Concatenate with existing df along rows
dir_folder = '/Users/pandabear/springboard/CapstoneTwoProject/data/interim/'
existing_df = pd.read_csv(dir_folder + 'listing_df_parsed.csv',index_col='listing_id')

combined_df = pd.concat([existing_df, new_listing_df], axis=0)

In [7]:
existing_df.shape

(2207887, 13)

In [8]:
combined_df.shape

(2507463, 13)

In [9]:
combined_df.to_csv(r'/Users/pandabear/springboard/CapstoneTwoProject/data/interim/listing_df_parsed.csv', header=True)
print('Saved to csv file')

Saved to csv file
